In [17]:
import requests
from math import sin, cos, sqrt, atan2, radians
import pandas as pd
import numpy as np
import sqlite3
import json
import traceback
from sqlite3 import IntegrityError
import csv

In [18]:
def get_distance(lat1, lon1, lat2, lon2):
    
    # approximate radius of earth in km
    R = 6373.0
    Miles_per_Km = 0.621371

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c * Miles_per_Km

    return distance

In [19]:
def find_airports(lat, long, airports, r):
    ret = []
    best = r
    for index, row in airports.iterrows():
        distance = get_distance(lat, long, row['Latitude'], row['Longitude'])
        if distance <= r:
#             print(row)
            data = {'distance':distance,'airport':row['IATA']}
            if distance < best:
                ret.insert(0,data)
                best = distance
            else:
                ret.append(data)
    return ret

In [20]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [21]:
def extract_event_json(event):
    try:
        name = event['name']
        event_id = event['id']

        venue = event['_embedded']['venues'][0]
        location = {'address':venue['address']['line1'],'city':venue['city']['name'],'state':venue['state']['stateCode'],
                'latlong':[float(venue['location']['latitude']),float(venue['location']['longitude'])]}

        latlong = location['latlong']
        airports = find_airports(latlong[0], latlong[1], us_airports, 20)
        event['airports'] = airports
    except KeyError:
        print('KeyError')
#         traceback.print_exc()
        event = {}

    return event


In [22]:
def execute_query(q,val,cur,conn):
    try:
        cur.execute(q, val)
        conn.commit()
    except IntegrityError:
        print('Found a duplicate already in the DB.')
#         traceback.print_exc()


In [23]:
def get_all_events(page_number=0, classification='music'):
    remaining_pages = True
    total_pages = 0
    page_count = 0
    ret = []
    conn = create_connection('trav2.db')
    cur = conn.cursor()
    while remaining_pages:
        try:
            URL = f"https://app.ticketmaster.com/discovery/v2/events.json"
            PARAMS = {'apikey':'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page':page_number, 
                      'countryCode':'US', 'classificationName':classification,'size':100} 
            print(PARAMS)
            r = requests.get(url = URL, params = PARAMS) 
            data = r.json()
            page = data['page']
            size = page['size']
            page_number = page['number']
            total_pages = page['totalPages']
            events = data['_embedded']['events']
            print(f'page: {page_number}/{total_pages}')
            #         print(len(events))
            for event in events:
                event_json = extract_event_json(event)
                if len(event_json) > 0:
                    name = event_json['name']
                    if 'priceRanges' in event_json:
            #             print(name)
                        line = (event_json['id'], name, classification, json.dumps(event_json))
                        execute_query('insert into raw_events(event_id, name, classification, json) values(?,?,?,?)', line,cur,conn)
        except KeyError:
            traceback.print_exc()
        except IntegrityError:
            traceback.print_exc()

        remaining_pages = ( page_number * size ) < 500
        page_number += 1
        
    conn.close()

 

In [24]:
airports = pd.read_csv("airports.csv").rename(columns=lambda x: x.strip())
us_airports = airports.loc[airports['Country'] == 'United States']
get_all_events(0,'sports')

{'apikey': 'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page': 0, 'countryCode': 'US', 'classificationName': 'sports', 'size': 100}
page: 0/37
{'apikey': 'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page': 1, 'countryCode': 'US', 'classificationName': 'sports', 'size': 100}
page: 1/37
{'apikey': 'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page': 2, 'countryCode': 'US', 'classificationName': 'sports', 'size': 100}
page: 2/37
{'apikey': 'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page': 3, 'countryCode': 'US', 'classificationName': 'sports', 'size': 100}
page: 3/37
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
{'apikey': 'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page': 4, 'countryCode': 'US', 'classificationName': 'sports', 'size': 100}
page: 4/37
Found a duplicate already in the DB.
{'apikey': 'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page': 5, 'country

In [25]:
def get_event_names(from_raw=True):
    conn = create_connection('trav2.db')
    cur = conn.cursor()
    if from_raw:
        cur.execute(f'select distinct name from raw_events order by name')
    else:
        cur.execute(f'select distinct name from my_events order by name')
    names = cur.fetchall()
    conn.close()
    for name in names:
        print(name[0])

In [26]:
def store_event(event, my_location):
    conn = create_connection('trav2.db')
    cur = conn.cursor()
    
    dates = event['dates']['start']
    event_id = event['id']
    sales = event['sales']['public']

    prices = event['priceRanges']
    venue = event['_embedded']['venues'][0]
    location = {'address':venue['address']['line1'],'city':venue['city']['name'],'state':venue['state']['stateCode'],
                'latlong':[float(venue['location']['latitude']),float(venue['location']['longitude'])]}

    event_date = dates['localDate']
    event_time = dates['localTime']
    price_range = [prices[0]['min'],prices[0]['max']]
    distance = get_distance(my_location[0],my_location[1],location['latlong'][0],location['latlong'][1])
    latlong = location['latlong']
    airports = find_airports(latlong[0], latlong[1], us_airports, 20)


    event_line ={'event_id':event_id,'event_date': event_date, 
                 'event_time':event_time, 'price_range':price_range, 
                 'location':location, 'distance':distance, 'airports':airports}

    val = (event_id,
           event['name'],
           event_date,
           event_time,
           price_range[0],
           price_range[1],
           location['address'],
           location['city'],
           location['state'],
           latlong[0],
           latlong[1],
           distance,
           my_location[0],
           my_location[1],
           my_location[2],
           my_location[3])

    execute_query('INSERT INTO my_events(id,name,date,time,price_low,price_high,street_address,city,state,lat,long,distance,my_lat,my_long,my_city,my_state) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)',
                  val,cur,conn)

    for airport in airports:
        val2 = (event_id ,airport['airport'],airport['distance'])
        execute_query('insert into airports(event_id, airport, distance) values(?,?,?)',val2,cur,conn)

    conn.close()


In [27]:
def select_events(keyword, my_location):
    conn = create_connection('trav2.db')
    cur = conn.cursor()
    cur.execute(f'select * from raw_events where name like "%{keyword}%"')
    events = cur.fetchall()
    conn.close()
    
    for event in events:
        name = event[1]
#         print(name)
        event_json = json.loads(event[3])
        store_event(event_json, my_location)    

In [28]:
def get_event_names(from_raw=True):
    conn = create_connection('trav2.db')
    cur = conn.cursor()
    if from_raw:
        cur.execute(f'select distinct name from raw_events order by name')
    else:
        cur.execute(f'select distinct name from my_events order by name')
    names = cur.fetchall()
    conn.close()
    for name in names:
        print(name[0])

In [29]:
get_event_names(True)

146th Kentucky Derby - Infield & Paddock General Admission Single Day
31st Southern Heritage Classic: Jackson State v Tennessee State
AEW Presents "Dynamite"
AEW Presents 'DYNAMITE' 
All Elite Wrestling
Atlanta Braves vs. Chicago Cubs
Atlanta Braves vs. Los Angeles Angels
Atlanta Braves vs. Los Angeles Dodgers
Atlanta Falcons vs. Carolina Panthers
Atlanta Falcons vs. Chicago Bears
Atlanta Falcons vs. Cincinnati Bengals
Atlanta Falcons vs. Denver Broncos
Atlanta Falcons vs. Detroit Lions
Atlanta Falcons vs. Las Vegas Raiders
Atlanta Falcons vs. New Orleans Saints
Atlanta Falcons vs. Seattle Seahawks
Atlanta Falcons vs. Tampa Bay Buccaneers
Baltimore Orioles vs. New York Yankees
Baltimore Ravens vs. Carolina Panthers
Baltimore Ravens vs. Cincinnati Bengals
Baltimore Ravens vs. Cleveland Browns
Baltimore Ravens vs. Dallas Cowboys
Baltimore Ravens vs. Jacksonville Jaguars
Baltimore Ravens vs. Kansas City Chiefs
Baltimore Ravens vs. New York Giants
Baltimore Ravens vs. Pittsburgh Steelers
B

In [30]:
preferred_event_names = [
    'New York Yankees']

likely_departure_points = [
    [40.790152, -111.979038, 'Salt Lake City', 'UT'],
    [36.085044, -115.149928, 'Las Vegas', 'NV'],
    [39.851704, -104.673811, 'Denver', 'CO']]

for event_name in preferred_event_names:
    for location in likely_departure_points:
        print(f'Getting {event_name} from {location[2]}')
        select_events(event_name,location)

Getting New York Yankees from Salt Lake City
Getting New York Yankees from Las Vegas
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already i

Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
F

Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
F

Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
F

Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
F

...

Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
F

Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
F

Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
F

Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
F

Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
Found a duplicate already in the DB.
F

In [15]:
def get_my_events(order_by):
    conn = create_connection('trav2.db')
    cur = conn.cursor()
    cur.execute(f'select name, price_low, price_high, distance, date, city, state, my_city from my_events {order_by}')
    out = cur.fetchall()
    conn.close()
    return out

my_events = get_my_events("order by price_low, date")
with open('my_events.csv', mode='w') as events_file:
    event_writer = csv.writer(events_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for event in my_events:
        print(f"{event[0]:<30} ${event[1]:<7.2f} to ${event[2]:<8.2f} {event[3]} {event[4]}, {event[5]}, {event[6]}, {event[7]}")
        event_writer.writerow(event)


New York Yankees v. Tampa Bay Rays $10.00   to $510.00   1974.5724284297498 2020-08-17, Bronx, NY, Salt Lake City
New York Yankees v. Tampa Bay Rays $10.00   to $510.00   2234.2030761664796 2020-08-17, Bronx, NY, Las Vegas
New York Yankees v. Tampa Bay Rays $10.00   to $510.00   1612.8071713868412 2020-08-17, Bronx, NY, Denver
New York Yankees v. Baltimore Orioles $10.00   to $510.00   1974.5724284297498 2020-08-26, Bronx, NY, Salt Lake City
New York Yankees v. Baltimore Orioles $10.00   to $510.00   2234.2030761664796 2020-08-26, Bronx, NY, Las Vegas
New York Yankees v. Baltimore Orioles $10.00   to $510.00   1612.8071713868412 2020-08-26, Bronx, NY, Denver
New York Yankees v. Minnesota Twins $12.00   to $510.00   1974.5724284297498 2020-05-28, Bronx, NY, Salt Lake City
New York Yankees v. Minnesota Twins $12.00   to $510.00   2234.2030761664796 2020-05-28, Bronx, NY, Las Vegas
New York Yankees v. Minnesota Twins $12.00   to $510.00   1612.8071713868412 2020-05-28, Bronx, NY, Denver
S